# Consignes

Intégrer le fichier USvideos.csv qui représente un ensemble de 8000 vidéos Youtube. 

Merger le fichier US_category_id.json pour récupérer le nom des catégories. Il conviendra de bien spécifier l'ID du document.


# Questions 
- 1) Récupérer toutes les vidéos de la chaîne Apple.
- 2) Compter le nombre de catégories différentes
- 3) Si vous ne l'avez pas déjà fait, découper les tags en listes et mettre à jour les tags de chacun des documents avec une requête update.
- 4) Récupérer les vidéos les plus vues.
- 5) Compter le nombre moyen de vues en fonction de la catégorie.
- 6) Récupérer les chaines Youtube avec la plus grande moyenne de likes.

In [3]:
import pandas as pd
import pymongo

In [4]:
client = pymongo.MongoClient()
database = client['exercices']
collection = database['youtube']

In [3]:
df_youtube = pd.read_csv("./data/USvideos.csv")
df_youtube.head(2)

,video_id,title,channel_title,category_id,tags,views,likes,dislikes,comment_total,thumbnail_link,date
0,XpVt6Z1Gjjo,1 YEAR OF VLOGGING -- HOW LOGAN PAUL CHANGED Y...,Logan Paul Vlogs,24,logan paul vlog|logan paul|logan|paul|olympics...,4394029,320053,5931,46245,https://i.ytimg.com/vi/XpVt6Z1Gjjo/default.jpg,13.09
1,K4wEI5zhHB0,iPhone X — Introducing iPhone X — Apple,Apple,28,Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lig...,7860119,185853,26679,0,https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg,13.09


## Question 0

### Netoyer les données

In [4]:
df_youtube['date'] = pd.to_datetime(df_youtube['date'], errors='coerce')
df_youtube = df_youtube.dropna(subset=['date'])


df_youtube.rename(columns={'video_id':'_id'}, inplace=True)

df_youtube = df_youtube.drop_duplicates(subset='_id', keep='last')

l_d =df_youtube.to_dict(orient='records')

import json
with open('./data/US_category_id.json', 'r') as f:
    categ = json.load(f)

for video in l_d:
    for cat in categ['items']:
        if video['category_id'] == int(cat['id']):
            video['category_name'] = cat['snippet']['title']

### Importer les données

In [6]:
from pymongo import MongoClient

client = MongoClient("localhost:27017")
db_ytb = client.ytb
collection_youtube = db_ytb['youtube'] 

collection_youtube.insert_many(l_d)

## Question 1  

In [48]:
cur = collection_youtube.find({"channel_title":"Apple"})
for video_apple in cur:
    print(video_apple)
    

{'_id': 'K4wEI5zhHB0', 'title': 'iPhone X — Introducing iPhone X — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple|iPhone 10|iPhone Ten|iPhone|Portrait Lighting|A11 Bionic|augmented reality|emoji|animoji|Face ID|Apple Pay|camera|smartphone', 'views': 19707391, 'likes': 381919, 'dislikes': 69465, 'comment_total': 0, 'thumbnail_link': 'https://i.ytimg.com/vi/K4wEI5zhHB0/default.jpg', 'date': datetime.datetime(1970, 1, 1, 0, 0), 'category_name': 'Science & Technology'}
{'_id': 'A9k88sMyiJM', 'title': 'Apple Watch Series 3 + Apple Music — Roll — Apple', 'channel_title': 'Apple', 'category_id': 28, 'tags': 'Apple|airpods|apple watch|apple watch series 3|apple watch cellular|apple watch 4g|apple music|40 million songs on your wrist|apple skateboard ad|apple train station ad|apple roll ad|apple watch skateboard ad|apple watch train station ad|apple watch roll ad|airpods skateboard ad|airpods train station ad|airpods roll ad|apple music skateboard ad|apple music train stati

## Question 2

In [110]:
cur = collection_youtube.distinct("category_name")
cpt = 0
for i in cur:
    cpt += 1
print(str(cpt) + ' différentes catégories')

16 différentes catégories


## Question 3

In [160]:
cur = collection_youtube.find()
for i in list(cur):
    #print(i['_id'])
    tags = i['tags'].split('|')
    result = collection_youtube.update_one({"_id":i['_id']}, {"$set":{"tags":tags}})
    result

## Question 4

In [165]:
cur = collection_youtube.find().sort("views",-1).limit(3)
print(list(cur))
    

[{'_id': 'MBdVXkSdhwU', 'title': "BTS (방탄소년단) 'DNA' Official MV", 'channel_title': 'ibighit', 'category_id': 10, 'tags': ['BIGHIT', '빅히트', '방탄소년단', 'BTS', 'BANGTAN', '방탄'], 'views': 41500672, 'likes': 2010366, 'dislikes': 78076, 'comment_total': 736179, 'thumbnail_link': 'https://i.ytimg.com/vi/MBdVXkSdhwU/default.jpg', 'date': datetime.datetime(1970, 1, 1, 0, 0), 'category_name': 'Music'}, {'_id': 'tt2k8PGm-TI', 'title': 'ZAYN - Dusk Till Dawn ft. Sia', 'channel_title': 'ZaynVEVO', 'category_id': 10, 'tags': ['Dusk Till Dawn', 'Pop', 'RCA Records Label', 'ZAYN feat. Sia'], 'views': 36323498, 'likes': 1431683, 'dislikes': 28049, 'comment_total': 100661, 'thumbnail_link': 'https://i.ytimg.com/vi/tt2k8PGm-TI/default.jpg', 'date': datetime.datetime(1970, 1, 1, 0, 0), 'category_name': 'Music'}, {'_id': 'LunHybOKIjU', 'title': 'Eminem Rips Donald Trump In BET Hip Hop Awards Freestyle Cypher', 'channel_title': 'BETNetworks', 'category_id': 24, 'tags': ['BET', 'BET Networks', 'BET Music', 'Bl

## Question 5

In [170]:
cur = collection_youtube.aggregate([{"$group" : {"_id" : "$category_name", "averageViews" : {"$avg" : "$views"}}}])
list(cur)


[{'_id': 'Education', 'averageViews': 550585.2659574468},
 {'_id': 'Music', 'averageViews': 1442240.943283582},
 {'_id': 'Film & Animation', 'averageViews': 966547.2871287129},
 {'_id': 'Autos & Vehicles', 'averageViews': 622727.8648648649},
 {'_id': 'Gaming', 'averageViews': 777213.0357142857},
 {'_id': 'Pets & Animals', 'averageViews': 633906.4210526316},
 {'_id': 'Nonprofits & Activism', 'averageViews': 681178.2},
 {'_id': 'Sports', 'averageViews': 618074.711409396},
 {'_id': 'Comedy', 'averageViews': 1317200.7289719626},
 {'_id': 'Entertainment', 'averageViews': 1059427.7735849055},
 {'_id': 'News & Politics', 'averageViews': 533575.1262626263},
 {'_id': 'Shows', 'averageViews': 8726.0},
 {'_id': 'Howto & Style', 'averageViews': 556002.6679389313},
 {'_id': 'Science & Technology', 'averageViews': 861563.6158940397},
 {'_id': 'Travel & Events', 'averageViews': 426492.22222222225},
 {'_id': 'People & Blogs', 'averageViews': 892992.28515625}]

## Question 6 

In [7]:
cur = collection_youtube.aggregate([
    {"$group" : {"_id" : "$channel_title", "averageLikes" : {"$avg" : "$likes"}}},
    {"$sort" : {"averageLikes" : -1}},
    {"$limit":5}
])
list(cur)

[{'_id': 'ibighit', 'averageLikes': 1533457.0},
 {'_id': 'ZaynVEVO', 'averageLikes': 1431683.0},
 {'_id': 'BETNetworks', 'averageLikes': 982842.0},
 {'_id': 'melanie martinez', 'averageLikes': 921577.0},
 {'_id': 'jypentertainment', 'averageLikes': 793387.0}]